## Homework 2 Solution Template
### CSCI 4270 / 6270
### Due: February 5, 2024

In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import math

## Problem 1

In [211]:
def print_matrix(name, A):
    print(f'Matrix {name}:')
    for i in range(A.shape[0]):
        row = A[i, :]
        print(f'{row[0]:.1f}', end='')
        for j in range(1, row.shape[0]):
            print(f', {row[j]:.1f}', end='')
        print()
    print()

In [222]:
def parse_matrix(matrix: np.ndarray) -> tuple[tuple[float], np.ndarray, tuple[float]]:
  """
  Given {matrix}, the
  - first row is extracted to create the rotation matrix, R
  - second row is extracted to create the translation vector, t_aug
  - third row is extracted such that it is a tuple of camera measurements, camera_metrics

  Args:
    matrix (np.ndarray): Matrix to be parsed for camera matrix parameters

  Returns:
    tuple[
      tuple[float] -> rotation degrees for x,y, and z axes
      np.ndarray -> translation vector
      tuple[float] -> camera metrics such as focal length and optical axis
  """
  degrees_across_axes: tuple[float] = tuple(matrix[0])
  t_aug: np.ndarray = np.array(matrix[1]).reshape(-1,1) # Updated such that it is a column vector
  camera_metrics: tuple[float] = tuple(matrix[2])
  return degrees_across_axes, t_aug, camera_metrics

def create_rotation_matrix(degrees_across_axes: tuple[float], translation_vector: np.ndarray) -> tuple[np.ndarray, tuple[np.ndarray]]:
  """
  Derives R = RxRyRz, by using the respective degrees across the axes

  Args:
    degrees_across_axes (tuple[float]): The degrees of the rotation for x, y, and z

  Returns:
    tuple[
      np.ndarray -> Rotation matrix,
      tuple[
        np.ndarray -> Rx,
        np.ndarray -> Ry,
        np.ndarray -> Rz
      ]
    ]
  """
  thetax, thetay, thetaz = degrees_across_axes
  thetax, thetay, thetaz = math.radians(thetax),math.radians(thetay), math.radians(thetaz)
  Rx: np.ndarray = np.array([
      [1, 0, 0],
      [0, math.cos(thetax), -math.sin(thetax)],
      [0, math.sin(thetax), math.cos(thetax)]
  ])
  Ry: np.ndarray = np.array([
      [math.cos(thetay), 0, math.sin(thetay)],
      [0, 1, 0],
      [-math.sin(thetay), 0, math.cos(thetay)]
  ])
  Rz: np.ndarray = np.array([
      [math.cos(thetaz), -math.sin(thetaz), 0],
      [math.sin(thetaz), math.cos(thetaz), 0],
      [0, 0, 1]
  ])
  R: np.ndarray = Rx @ Ry @ Rz # Rotation metrics
  print_matrix(name="R", A=R)

  return R, (Rx, Ry, Rz)


def create_intrisic_matrix(f: float, pixel_dimensions: tuple[float], optical_center: tuple[int], unit_conversion: float = 10e-4) -> tuple[np.ndarray, tuple[float]]:
  """
  Derives the intrisic matrix or K matrix.

  Args:
    f (float): Focal length of camera
    pixel_dimensions (tuple[float]): How big each pixel is in the x and y directions
    optical_center (tuple[int]): Where the camera's center is located w.r.t pixel coordinates
    unit_conversion (float): Conversion from microns to {specified}

  Returns:
    tuple[
      np.ndarray -> Matrix with intrisic parameters
      tuple[float] -> scaling factor in x and y
    ]
  """
  sx: float = f / (pixel_dimensions[1] * unit_conversion)
  sy: float = f / (pixel_dimensions[0] * unit_conversion)

  K: np.ndarray = np.array([
    [sx, 0, optical_center[1]],
    [0, sy, optical_center[0]],
    [0, 0, 1]
  ])
  print_matrix(name="K",A=K)

  return K, (sx, sy)

def create_camera_matrix(R: np.ndarray, K: np.ndarray, translation_vector: np.ndarray) -> np.ndarray:
  """
  Generates camera matrix, M, using the rotation-translational matrix and intrisic parameter matrix

  Args:
    R (np.ndarray): Rotational-translational matrix
    K (np.ndarray): Intrisic parameter matrix
    translation_vector (np.ndarray): Vector that moves camera in x,y,z

  Returns:
    np.ndarray -> camera matrix
  """
  M: np.ndarray = K @ np.concatenate((R.T, -R.T @ translation_vector), axis=1)
  print_matrix(name="M",A=M)
  return M

def apply_camera_matrix(M: np.ndarray, datapoint: np.ndarray) -> tuple[float]:
  """
  Computes image locations, [u', v', w'] using the camera matrix and datapoint
    [u', v', w'] = M @ datapoint

  Args:
    M (np.ndarray): Camera matrix
    datapoint (np.ndarray): Datapoint to be converted to image location

  Returns:
    tuple[float] -> datapoint in image location
  """
  datapoint_aug: np.ndarray = (np.append(datapoint, (1))).reshape(-1, 1)
  image_location: np.ndarray = M @ datapoint_aug
  up, vp, wp = tuple(image_location)
  up, vp, wp = up[0], vp[0], wp[0]

  u: float = up / wp
  v: float = vp / wp

  return (u, v)

def check_is_inside(point: tuple[int], bounds: tuple[int]) -> bool:
  """
  Returns true iff the point, {point}, is within the given bounds, {bounds}

  Args:
    point (tuple[int]): row, col coordinates of a point in an image
    bounds (tuple[int]): max_rows, max_cols in a given image
  """
  y, x = point
  max_rows, max_cols = bounds
  min_rows, min_cols = 0,0

  return (y <= max_rows and y >= min_rows) and (x <= max_cols and x >= min_cols)

def project_point(R: np.ndarray, t: np.ndarray, datapoint: np.ndarray) -> np.ndarray:
  """
  Adjust camera's points with R and t

  Args:
    R (np.ndarray): Rotation matrix
    t (np.ndarray): translational matrix
    datapoint (np.ndarray): Given point in the data

  Returns:
    np.ndarray -> updated camera point, post-projection
  """
  return R.T @ (datapoint - t)

def display_transformation(M: np.ndarray, data: np.ndarray, R: np.ndarray, t: np.ndarray) -> None:
  """
  Applies the camera matrix to each datapoint and it outputs the results as required.

  Args:
    M (np.ndarray): Camera matrix
    data (np.ndarray): Data
    R: Rotation matrix
    t: translation vector
  """
  print("Projections:")
  x: float
  y: float
  z: float
  u: float
  v: float
  is_inside: bool
  inside_text: str
  visible: list[int] = []
  hidden: list[int] = []
  projected_point: np.ndarray

  for index, datapoint in enumerate(data):
    x, y, z = tuple(datapoint)
    u, v = apply_camera_matrix(M=M, datapoint=datapoint)
    is_inside = check_is_inside(point=(v,u), bounds=(4000, 6000))
    if (is_inside): inside_text = "inside"
    else: inside_text = "outside"

    print(f"{index}: point {x:.1f} {y:.1f} {z:.1f} ==> {v:.1f} {u:.1f} {inside_text}")

    projected_point = project_point(R=R, t=t, datapoint=datapoint.reshape(-1,1))
    if (projected_point[2] > 0): visible.append(index)
    else: hidden.append(index)

  visible.sort()
  hidden.sort()

  print("visible: ", end="")
  for i in visible:
    print(i, end=" ")

  print()

  print("hidden: ", end="")
  for i in hidden:
    print(i, end=" ")

In [213]:
params = [
    [0.0, 0.0, 0.0],
    [0.0, 0.0, 10.0],
    [15, 10, 2001, 2995]
]
points = [
    [10, 5, 100],
    [0, 0, 0.5],
    [-30, 10, -20],
    [20, 15, 20]
]

In [223]:
degrees_across_axes, translational_vector, camera_metrics = parse_matrix(matrix=params)
f, d, ic, jc = camera_metrics
R, _ = create_rotation_matrix(degrees_across_axes=degrees_across_axes, translation_vector=translational_vector)
K, scaling_factors = create_intrisic_matrix(f=f, pixel_dimensions=(d,d), optical_center=(ic,jc))
M = create_camera_matrix(R=R, K=K, translation_vector=translational_vector)
points: np.ndarray = np.array(points)
image_location: np.ndarray = apply_camera_matrix(M=M, datapoint=points[0,:])

display_transformation(M=M, data=points, R=R, t=translational_vector)

Matrix R:
1.0, 0.0, 0.0
0.0, 1.0, 0.0
0.0, 0.0, 1.0

Matrix K:
1500.0, 0.0, 2995.0
0.0, 1500.0, 2001.0
0.0, 0.0, 1.0

Matrix M:
1500.0, 0.0, 2995.0, -29950.0
0.0, 1500.0, 2001.0, -20010.0
0.0, 0.0, 1.0, -10.0

Projections:
0: point 10.0 5.0 100.0 ==> 2084.3 3161.7 inside
1: point 0.0 0.0 0.5 ==> 2001.0 2995.0 inside
2: point -30.0 10.0 -20.0 ==> 1501.0 4495.0 inside
3: point 20.0 15.0 20.0 ==> 4251.0 5995.0 outside
visible: 0 3 
hidden: 1 2 

In [225]:
params2 = [
    [15.0, -45.0, 10.0],
    [4.0, 30.0, 10.0],
    [12, 12, 1998, 3005]
]
points2 = [
    [100, 15, 90],
    [-100, 800, 1500],
    [10, -500, -500],
    [-30, 10, 20]
]

degrees_across_axes, translational_vector, camera_metrics = parse_matrix(matrix=params2)
f, d, ic, jc = camera_metrics
R, _ = create_rotation_matrix(degrees_across_axes=degrees_across_axes, translation_vector=translational_vector)
K, scaling_factors = create_intrisic_matrix(f=f, pixel_dimensions=(d,d), optical_center=(ic,jc))
M = create_camera_matrix(R=R, K=K, translation_vector=translational_vector)
points2: np.ndarray = np.array(points2)
image_location: np.ndarray = apply_camera_matrix(M=M, datapoint=points2[0,:])

display_transformation(M=M, data=points2, R=R, t=translational_vector)

Matrix R:
0.7, -0.1, -0.7
-0.0, 1.0, -0.2
0.7, 0.1, 0.7

Matrix K:
1000.0, 0.0, 3005.0
0.0, 1000.0, 1998.0
0.0, 0.0, 1.0

Matrix M:
-1428.5, -562.5, 2770.0, -5112.7
-1535.6, 617.4, 1500.9, -27388.2
-0.7, -0.2, 0.7, 1.5

Projections:
0: point 100.0 15.0 90.0 ==> 3487.2 -8851.4 outside
1: point -100.0 800.0 1500.0 ==> 3021.6 4043.8 inside
2: point 10.0 -500.0 -500.0 ==> 4311.3 4394.6 outside
3: point -30.0 10.0 20.0 ==> 1589.0 2534.4 inside
visible: 1 3 
hidden: 0 2 

In [ ]:
params3 = [
    [-16.0, 10.0, 50.0],
    [25, -12, 50],
    [9, 9, 1000, 1400]
]

degrees_across_axes, translational_vector, camera_metrics = parse_matrix(matrix=params3)
f, d, ic, jc = camera_metrics
R, _ = create_rotation_matrix(degrees_across_axes=degrees_across_axes, translation_vector=translational_vector)
K, scaling_factors = create_intrisic_matrix(f=f, pixel_dimensions=(d,d), optical_center=(ic,jc))
print_matrix("K",K)
M = create_camera_matrix(R=R, K=K, translation_vector=translational_vector)
print_matrix("M",M)

In [ ]:
def p1_camera():
  pass

In [ ]:
"""
Problem 1, Test 1. This is a very simple test that could be easily checked by hand.
"""
params = [
    [0.0, 0.0, 0.0],
    [0.0, 0.0, 10.0],
    [15, 10, 2001, 2995]
]
points = [
    [10, 5, 1000, 0, 0.5],
    [-30, 10, -20],
    [20, 15, 20]
]

p1_camera(params, points)


In [ ]:
'''
Problem 1, Test 2
'''
params = [
    [15.0, -45.0, 10.0],
    [4.0, 30.0, 10.0],
    [12, 12, 1998, 3005]
]
points = [
    [100, 15, 90],
    [-100, 800, 1500],
    [10, -500, -500],
    [-30, 10, 20]
]
p1_camera(params, points)


In [ ]:
'''
Problem 1, Test 3
'''
params = [
    [-16.0, 10.0, 50.0],
    [25, -12, 50],
    [9, 9, 1000, 1400]
]
points = [
    [100, 15, 90],
    [-100, 800, 1500],
    [10, -500, -500],
    [-30, 10, 20]
]
p1_camera(params, points)

## Problem 2

In [7]:
def load_points(fn):
    '''
    Input: a path to a file containing x, y points, one point per line.
    Returns: two-d np array where each row contains an x, y point
    '''
    f = open(fn, 'r')
    pts = []
    for line in f:
        line = line.strip().split()
        x, y = float(line[0]), float(line[1])
        pts.append([x, y])
    pts = np.array(pts)
    f.close()
    return pts

In [8]:
pts1_filepath = 'data/p2_pts1_in.txt'
display(load_points(fn=pts1_filepath))

array([[-23.906483,  15.170165],
       [-22.430193,   3.459445],
       [-23.773563,  14.673693],
       [-21.568955,  26.063764],
       [-13.567062,   9.954461],
       [-21.361401,  13.961469],
       [-18.074484,  12.428326],
       [-20.915688,  13.889777],
       [-16.720927,   9.099601],
       [-23.159744,  15.962732],
       [-19.792109,  14.082285],
       [-15.261347,  16.567496],
       [-15.94603 ,  12.087928],
       [-21.628379,  14.955305],
       [-13.019561,  -1.422918],
       [-26.745567,  19.436875],
       [-20.319142,  12.183828],
       [-24.878528,  16.680258],
       [-11.926546,   8.416869],
       [-23.671746,  16.921441],
       [-26.787085,   3.62415 ],
       [-28.315417,  12.052977],
       [-24.873056,  23.169655],
       [-25.064775,  17.436202],
       [-11.707042,  10.204407],
       [-16.264291,  13.082468],
       [-21.106707,  28.504766],
       [-25.123917,  17.251031],
       [-21.298139,  29.13411 ],
       [-19.233579,  14.685527]])

In [29]:
def perform_RANSAC(data: np.ndarray, num_samples: int, tau:float, seed: int=42) -> tuple[tuple[float], float]:
  """
  Given a set of points, an optimal line is fit using RANSAC

  Args:
    data (np.ndarray): Dataset
    num_samples (int): maximum number of samples to take
    tau (float): Distance threshold
    seed (int): Seed for np random

  Returns:
    tuple[
      tuple[
        float -> optimal a,
        float -> optimal b,
        float -> optimal c,
      ]
      float -> most number of inlier
      ]
  """
  np.random.seed(seed=seed)

  N: int = len(data)

  iteration: int = 0 # Current iteration of the sampling
  sample_indices: np.ndarray # Indices for two sample points
  sample_point1: np.ndarray
  sample_point2: np.ndarray
  k_max: int = 0
  theta_max: float
  rho_max: float

  # Randomly selected coordinates
  x1: float
  x2: float
  y1: float
  y2: float

  # Helper functions
  delta_vector: np.ndarray
  compute_a: float = lambda theta: math.cos(theta)
  compute_b: float = lambda theta: math.sin(theta)
  compute_c: float = lambda rho: -rho
  compute_average: float = lambda items: sum(items) / len(items)
  compute_angle: float = lambda delta_vector: math.atan2(-delta_vector[0], delta_vector[1])
  compute_distance: float = lambda a, b, x, y, c: np.abs(a*x + b*y + c)
  compute_rho: float = lambda a, b, x, y: a * x + b * y

  inlier_distances: list[float] = []
  outlier_distances: list[float] = []
  num_inliers: int

  # Line parameters at current iteration
  a_i: float
  b_i: float
  c_i: float
  theta_i: float
  rho_i: float

  while iteration < num_samples:
    sample_indices = np.random.randint(0, N, 2)

    if (sample_indices[0] == sample_indices[1]): # Two generated indices are equal
      iteration += 1
      continue
    # Get datapoints for iteration
    sample_point1 = data[sample_indices[0], :]
    sample_point2 = data[sample_indices[1], :]
    y1, x1 = tuple(sample_point1)
    y2, x2 = tuple(sample_point2)

    delta_vector = np.array([x2 - x1, y2 - y1])
    theta_i = compute_angle(delta_vector=delta_vector)
    a_i = compute_a(theta=theta_i)
    b_i = compute_b(theta=theta_i)
    rho_i = compute_rho(a=a_i, b=b_i, x=x1,y=y1)
    c_i = compute_c(rho=rho_i)

    if (c_i > 0):
      a_i = -a_i
      b_i = -b_i
      c_i = -c_i

    num_inliers = 0
    for datapoint in data:
      distance = compute_distance(a=a_i, b=b_i, c=c_i, x=datapoint[1], y=datapoint[0])
      if (distance < tau):
        inlier_distances.append(distance)
        num_inliers += 1
        continue
      outlier_distances.append(distance)

    if (num_inliers > k_max):
      k_max = num_inliers
      theta_max, rho_max = theta_i, rho_i
      a_max, b_max, c_max = a_i, b_i, c_i

      print(f"Sample {iteration}:")
      print(f"indices ({sample_indices[0]},{sample_indices[1]})")
      print(f"line ({a_i:.3f},{b_i:.3f}, {c_i:.3f})")
      print(f"inliers {num_inliers}")
      print()

    iteration += 1

  print(f"avg inlier dist {compute_average(items=inlier_distances):.2f}")
  print(f"avg outlier dist {compute_average(items=outlier_distances):.2f}")

  return (a_max, b_max, c_max, theta_max, rho_max), k_max

In [4]:
filepath = 'data/p2_pts1_in.txt'
num_samples = 25
tau = 2.5
seed = 999

In [5]:
def p2_ransac(filepath: str, num_samples: int, tau: float, seed:int = 42) -> None:
  """
    Reads in data and fits an optimal line to the data using RANSAC

    Args:
      filepath (str): Where to find dataset
      num_samples (int): maximum number of samples to take
      tau (float): Distance threshold
      seed (int): Seed for np random

    Returns:
      tuple[
        tuple[
          float -> optimal a,
          float -> optimal b,
          float -> optimal c,
          float -> optimal theta,
        ]
        float -> most number of inlier
        ]
    """
  # Read in data
  data: np.ndarray = load_points(fn=filepath)
  perform_RANSAC(data=data, num_samples=num_samples, tau=tau, seed=seed)
  # return perform_RANSAC(data=data, num_samples=num_samples, tau=tau, seed=seed)


In [30]:
p2_ransac(filepath=filepath, num_samples=num_samples, tau=tau,seed=seed)

Sample 4:
indices (19,16)
line (-0.816,-0.578, -9.548)
inliers 2

avg inlier dist 1.07
avg outlier dist 29.61


In [ ]:
'''
Problem 2, Test 1
'''
filepath = 'data/p2_pts1_in.txt'
num_samples = 25
tau = 2.5
seed = 999
p2_ransac(filepath, num_samples, tau, seed)


In [ ]:
'''
Problem 2, Test 2
'''
filepath = 'data/p2_pts2_in.txt'
num_samples = 35
tau = 3.0
seed = 1232
p2_ransac(filepath, num_samples, tau, seed)


## Problem 3 (4270 Only)


In [3]:
'''
Utility for Problem 3
'''
import os
from os import sys

def get_images(img_dir: str) -> tuple[list[str], list[np.ndarray]]:
    start_cwd:str = os.getcwd()
    os.chdir(img_dir)
    img_name_list: list[str] = os.listdir('./')
    img_name_list = [name for name in img_name_list if 'jpg' in name.lower()]
    img_name_list.sort()

    img_list: list[np.ndarray] = []
    im: np.ndarray

    for i_name in img_name_list:
        im = cv2.imread(i_name, cv2.IMREAD_GRAYSCALE)
        if im is None:
            print('Could not open', i_name)
            sys.exit(0)
        img_list.append(im)

    os.chdir(start_cwd)
    return img_name_list, img_list

In [24]:
def calculate_image_energy(image: np.ndarray) -> float:
  """
  Using Gaussian Smooting, we find the energy value of the image.

  Args:
    image (np.ndarray): Image to be evaluated

  Returns:
    float -> image's energy
  """

  # Compute derivatives and apply aspects of Gaussian Smooting to images w.r.t x and y axes
  im_xdelta: np.ndarray = cv2.Sobel(image, cv2.CV_32F, 1, 0) # Gradients or differences of all pxels across the x-axis
  im_ydelta: np.ndarray =  cv2.Sobel(image, cv2.CV_32F, 0, 1) # Gradients or differences of all pxels across the y-axis
  M, N = image.shape

  gradient_magnitude_squared: np.ndarray = ((im_xdelta) ** 2 + (im_ydelta) ** 2)
  total_gradient_magnitude_squared: np.ndarray = np.sum(gradient_magnitude_squared) # Sums all gradients within the image
  normalized_total: np.ndarray = (1 / (M * N)) * total_gradient_magnitude_squared

  return normalized_total

def find_best_focused_image(images: list[np.ndarray]) -> tuple[np.ndarray, int, list[float]]:
  """
  Using Gaussian Smooting, we find the energy value of each image in {images} and declare the best focused image as the image with the highest energy value.

  Args:
    images (list[np.ndarray]): List of images to be evaluated

  Returns:
    tuple[
      np.ndarray -> image with the best focus,
      int -> index of image with the best focus w.r.t the list it is in
  """
  image_energies: list[np.ndarray] = [] # Entry 1 corresponds to Entry 1 in images
  E_image: float
  for image in images:
    E_image = calculate_image_energy(image=image)
    image_energies.append(E_image)

  best_focused_image_index: int = np.argmax(image_energies)
  best_focused_image: np.ndarray = image[best_focused_image_index]
  return best_focused_image,best_focused_image_index,image_energies

In [ ]:
evergreen_dir = 'data/evergreen'
names_of_images, images = get_images(img_dir=evergreen_dir)
best_focused_image, best_focused_image_index, image_energies = find_best_focused_image(images=images)

E_image: np.ndarray
for i, image_name in enumerate(names_of_images):
  E_image = image_energies[i]
  print(f"{image_name}: {E_image:.1f}")
print(f"Image {names_of_images[best_focused_image_index]} is best focused.")

In [26]:
def p3_best_focus(image_dir: str) -> tuple[np.ndarray, int]:
  """
  Finds and outputs best foucused image

  Args:
    image_dir (str): Directory where images can be found

  Returns:
      tuple[
        np.ndarray -> image with the best focus,
        int -> index of image with the best focus -- corresponding to the index - 1 of the the sorted images in {image_dir}
  """
  names_of_images, images = get_images(img_dir=image_dir)
  best_focused_image, best_focused_image_index, image_energies = find_best_focused_image(images=images)

  E_image: np.ndarray
  for i, image_name in enumerate(names_of_images):
    E_image = image_energies[i]
    print(f"{image_name}: {E_image:.1f}")
  print(f"Image {names_of_images[best_focused_image_index]} is best focused.")
  return best_focused_image, best_focused_image_index


In [ ]:
evergreen_dir = 'data/evergreen'
p3_best_focus(evergreen_dir)

In [ ]:
image_dir = 'data/evergreen'
p3_best_focus(image_dir)

In [ ]:
image_dir = 'data/branches'
p3_best_focus(image_dir)